# MLOps Hackathon

This hackathon is based on the [open-source Turbo Template](https://github.com/teamdatatonic/vertex-pipelines-end-to-end-samples).
Through this notebook series you'll get hands-on with the template and Google Cloud.
The hackathon is structured into the following exercises:

1. [Health check](./01_health_check.ipynb)
1. [Run pipelines](./02_run_pipelines.ipynb)
1. **[Promote model](./03_promote_model.ipynb) - this notebook**
1. [Challenge: Model monitoring](./04_monitoring_challenge.ipynb)
1. [Challenge: Real-time predictions](./05_realtime_challenge.ipynb)


## Introduction

In this notebook you'll use a recommended a Google Cloud architecture to promote your model from development to production.
You'll learn about pull requests, releases, Cloud Build, and Cloud Scheduler.

## Authenticate

Set your project ID and authenticate using your Google Account:

In [ ]:
VERTEX_PROJECT_ID = "my-project-id"
GOOGLE_ACCOUNT = "user@company.com"
! gcloud config set project {VERTEX_PROJECT_ID}
! gcloud config set account {GOOGLE_ACCOUNT}
! gcloud auth login

In [1]:
%cd ..

/home/jupyter/mlops-hackathon


## Staging: Test your recent changes

Remember the updated training container from the last notebook? Let's commit your changes!

**➡️ Exercise:** Submit your code changes:
1. Create a new branch `feat/new-model-<your name>`
1. Commit, go to github and open a pull request from your new branch to `develop` branch of the **same** repo
1. Add a new comment with the content `/gcbrun`
1. Don't merge your pull request, however, observe what's triggered by your comment

**➡️ Exercise:** Check your understanding:
- Do you know where the executed jobs are defined? (Hint: you're running pipelines in Cloud Build)
- In which Google Cloud project are the jobs executed? Do you know why this decision was made?

## Production: Promote your model

Let's recall this Google Cloud architecture:

![Architecture](../docs/images/architecture.png)

**➡️ Exercise:** 

1. Locate your production project in Google Cloud console
1. Navigate to Cloud Build and verify that it includes a trigger upon tag
1. Go to [the hackathon repository](https://github.com/teamdatatonic/mlops-hackathon)
1. Create a release from your branch and provide the tag name `v1.0.0-<your name>`

![Create release](./images/create_release.png)

**➡️ Exercise:** Go back to Cloud Build in the **production** Google Cloud project and monitor the new job. What's happening in the job? Once finished:

1. Go to Artifact Registry in the Google Cloud console and locate the new docker images and pipelines
2. Can you find the created artifacts by your release?

## Production: Schedule your pipeline

After successfully creating a release and thereby promoting your code and containers to the **production** environment, let's schedule your training pipeline so that you can continuously retrain your model whenever new training data becomes available.

Update the variables below before executing the next code cells.

In [2]:
tag = "v1.0.0-eugene"
location = "europe-west2"
project = "dt-sky-mlops-hackathon-prod"

In [4]:
import json

template_path = f"https://{location}-kfp.pkg.dev/{project}/vertex-pipelines/turbo-training-pipeline/{tag}"
display_name = f"turbo-training-pipeline-{tag}"
enable_caching = "false"

data = dict(
    template_path       = template_path,
    display_name        = display_name,
    enable_caching      = enable_caching,
    pipeline_parameters = dict(
        project = project,
        location = location,
        bq_location = "US",
        bq_source_uri = "bigquery-public-data.chicago_taxi_trips.taxi_trips",
        model_name = f"model-{tag}",
    )
)

print(json.dumps(data))

{"template_path": "https://europe-west2-kfp.pkg.dev/dt-sky-mlops-hackathon-prod/vertex-pipelines/turbo-training-pipeline/v1.0.0-eugene", "display_name": "turbo-training-pipeline-v1.0.0-eugene", "enable_caching": "false", "pipeline_parameters": {"project": "dt-sky-mlops-hackathon-prod", "location": "europe-west2", "bq_location": "US", "bq_source_uri": "bigquery-public-data.chicago_taxi_trips.taxi_trips", "model_name": "model-v1.0.0-eugene"}}


**➡️ Exercise:** Schedule the training pipeline using Cloud Scheduler:

1. Locate Cloud Scheduler in the **production** Google Cloud project and create a new job <p align="center">
  <img src="./images/scheduler_1.png" width="400"/>
</p>
2. Paste as the message body the generated JSON from above <p align="center">
  <img src="./images/scheduler_2.png" width="400"/>
</p>
3. Create the job and force a manual execution
4. Go to Vertex AI pipeline and observe whether your training pipeline is running

**➡️ Exercise:** Can you think about a better (e.g. automated) way of scheduling pipelines?

You've successfully tested, promoted, and scheduled your first pipeline in a production environment! 🎉 Now you're ready for the next exercise!